In [1]:
import os
os.chdir("..")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

from SSTImportanceSampling  import Preprocessor, ImportanceSampler, StormDepthProcessor

from production.utils import summarize_depths_by_return_period
from production.metrics import metrics
from production.plots import plot_return_period_summary, plot_two_return_period_summaries
from production.plots import plot_adaptive_evolution

from production.consequence_adaptive import AdaptParamsConsequence, AdaptiveMixtureConsequenceSampler,ConsequenceAdaptManager

In [ ]:
watershed_names = ["Trinity","Kanawha","Duwamish","Denton"]

In [ ]:
#Load data
watersheds = {}

for wname in watershed_names:
    ws = Preprocessor.load(
        config_path=f"/workspaces/Importance-Sampling-for-SST/data/1_interim/{wname}/config.json"
    )
    watersheds[wname] = ws

In [ ]:
#Full Monte Carlo summary
base_dir = "/workspaces/Importance-Sampling-for-SST/data/1_interim"
summary_filename = "fullmonte_summary.pq"

# --- load summaries ---
summaries = {}
for w in watershed_names:
    path = os.path.join(base_dir, w, summary_filename)
    df = pd.read_parquet(path)
    needed = {"RP","median_in","ci95_low_in","ci95_high_in"}
    if not needed <= set(df.columns):
        raise ValueError(f"{w}: summary missing columns {needed - set(df.columns)}")
    summaries[w] = df.sort_values("RP")

In [ ]:
watershed = watersheds["Denton"]

In [ ]:
params = AdaptParams(
    mu_x_n=watershed.watershed_stats["x"],
    mu_y_n=watershed.watershed_stats["y"],
    sd_x_n=watershed.watershed_stats["range_x"],
    sd_y_n=watershed.watershed_stats["range_y"],

    mu_x_w=watershed.domain_stats["x"],
    mu_y_w=watershed.domain_stats["y"],
    sd_x_w=watershed.domain_stats["range_x"]/np.sqrt(12),
    sd_y_w=watershed.domain_stats["range_y"]/np.sqrt(12),

    rho_n=0.0,      # narrow corr
    rho_w=0.0,      # wide corr
    mix=0.8,        # initial narrow weight

    # Optional tuning:
    lambda_mix=0.30,
    alpha=0.75,
    K_temper=1.0,
)

sampler = AdaptiveMixtureConsequenceSampler(
    data=watershed,
    params=params,
    precip_cube=watershed.cumulative_precip,
    seed=42
)

mgr = ConsequenceAdaptManager(
    sampler=sampler,
    run_dir="./Denton_consequence_run",
    run_name="Denton_conseq_adapt",
    watershed_name="Denton"
)

In [ ]:
# --- Quick peek at current params (optional) ---
asdict(mgr.current_params())

In [ ]:
iter_idx = 1               # bump this manually for the next iteration (2, 3, ...)
samples_per_iter = 1000
seed_for_iter = 12345      # or None to use internal RNG

req_paths = mgr.sample_and_write_request(
    iter_idx=iter_idx,
    samples_per_iter=samples_per_iter,
    seed=seed_for_iter
)
req_paths

In [ ]:
#sample_id, consequence  f"{run_dir}/iter_{iter_idx:03d}/to_hms.csv"

In [ ]:
consequence_csv_path = f"{run_dir}/iter_{iter_idx:03d}/consequences.csv"  # put your results there
result_paths = mgr.adapt_once_with_consequences(
    iter_idx=iter_idx,
    consequences_csv_path=consequence_csv_path,
    reward_col="consequence",
    threshold=0.0
)
result_paths

In [ ]:
pd.read_csv(Path(run_dir) / "history.csv")

In [ ]:
from consequence_adapt import to_parquet  # convenience wrapper

final_df = sampler.sample_batch(n=5000, iter_idx=999, seed=777)
to_parquet(final_df, Path(run_dir) / "final_draws.parquet")
final_df.head()